In [196]:
import re
import requests
import time, os
import itertools

from collections import defaultdict
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [256]:
columns = ['data.economy.budget.date', 'data.economy.budget.expenditures.units',
           'data.economy.budget.expenditures.value', 'data.economy.budget.revenues.units',
           'data.economy.budget.revenues.value', 'data.economy.budget_surplus_or_deficit.date',
           'data.economy.budget_surplus_or_deficit.percent_of_gdp', 'data.economy.exports.partners.by_country',
           'data.economy.exports.partners.date', 'data.economy.gdp.composition.by_end_use.date',
           'data.economy.gdp.composition.by_end_use.end_uses.exports_of_goods_and_services.units',
           'data.economy.gdp.composition.by_end_use.end_uses.exports_of_goods_and_services.value',
           'data.economy.gdp.composition.by_end_use.end_uses.household_consumption.value',
           'data.economy.household_income_by_percentage_share.highest_ten_percent.units',
           'data.economy.household_income_by_percentage_share.highest_ten_percent.value',
           'data.economy.household_income_by_percentage_share.lowest_ten_percent.units',
           'data.economy.household_income_by_percentage_share.lowest_ten_percent.value',
           'data.economy.industrial_production_growth_rate.annual_percentage_increase', 
           'data.economy.industrial_production_growth_rate.date', 'data.energy.electricity.access.date',
           'data.energy.electricity.access.total_electrification.value', 
           'data.energy.electricity.by_source.fossil_fuels.date',
           'data.energy.electricity.by_source.fossil_fuels.percent', 
           'data.energy.electricity.by_source.hydroelectric_plants.date',
           'data.energy.electricity.by_source.hydroelectric_plants.percent', 
           'data.energy.electricity.by_source.nuclear_fuels.date', 
           'data.energy.electricity.by_source.nuclear_fuels.percent',
           'data.energy.electricity.by_source.other_renewable_sources.date', 
           'data.energy.electricity.by_source.other_renewable_sources.percent']

In [257]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
url = 'https://iancoleman.io/explorer-cia-world-factbook/'

In [258]:
driver = webdriver.Chrome()
driver.get(url)

In [259]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [260]:
chooseFile = driver.find_element_by_xpath('//div/p[@class="dz-clickable"]')

In [261]:
#credit: Roman Konoval StackOverflow
#https://stackoverflow.com/questions/43382447/python-with-selenium-drag-and-drop-from-file-system-to-webdriver

'''
Uses JS_DROP_FILE script to enable selenium to drag and drop a file from 
local machine as input for website.
'''

JS_DROP_FILE = """
    var target = arguments[0],
        offsetX = arguments[1],
        offsetY = arguments[2],
        document = target.ownerDocument || document,
        window = document.defaultView || window;

    var input = document.createElement('INPUT');
    input.type = 'file';
    input.onchange = function () {
      var rect = target.getBoundingClientRect(),
          x = rect.left + (offsetX || (rect.width >> 1)),
          y = rect.top + (offsetY || (rect.height >> 1)),
          dataTransfer = { files: this.files };

      ['dragenter', 'dragover', 'drop'].forEach(function (name) {
        var evt = document.createEvent('MouseEvent');
        evt.initMouseEvent(name, !0, !0, window, 0, 0, 0, x, y, !1, !1, !1, !1, 0, null);
        evt.dataTransfer = dataTransfer;
        target.dispatchEvent(evt);
      });

      setTimeout(function () { document.body.removeChild(input); }, 25);
    };
    document.body.appendChild(input);
    return input;
"""

def drag_and_drop_file(drop_target, path):
    driver = drop_target.parent
    file_input = driver.execute_script(JS_DROP_FILE, drop_target, 0, 0)
    file_input.send_keys(path)

In [262]:
#credit to: Rob Hawkins from stackoverflow
#https://stackoverflow.com/questions/37979644/parse-beautifulsoup-element-into-selenium

def xpath_soup(element):
    """
    Generate xpath of soup element
    :param element: bs4 text or node
    :return: xpath as string
    """
    components = []
    child = element if element.name else element.parent
    for parent in child.parents:
        """
        @type parent: bs4.element.Tag
        """
        previous = itertools.islice(parent.children, 0, parent.contents.index(child))
        xpath_tag = child.name
        xpath_index = sum(1 for i in previous if i.name == xpath_tag) + 1
        components.append(xpath_tag if xpath_index == 1 else '%s[%d]' % (xpath_tag, xpath_index))
        child = parent
    components.reverse()
    return '/%s' % '/'.join(components) 

In [263]:
path = os.path.abspath("/Users/emilyng/Documents/chi20_ds15/curriculum/project-02/Proj-02-Data/factbook 2013-2018/2020-05-04_factbook.json")
drag_and_drop_file(chooseFile, path)

In [264]:
change_columns_button = driver.find_element_by_xpath('//p/button[@type="button"]').click()

In [265]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [266]:
def check_boxes(soup):
    '''
    Check all the boxes for features of interest (specified in 'columns').
    This method may result in some leak-over of features not explicitly specified in 'columns'
    list. This is due to the cases where the column is not one of the checkbox options
    
    check_boxes() will print out all features in 'columns' list that does not appear as a 
    check box option
    '''
    global columns
    for i in range(len(columns)):
        try:
            soup_element = soup.find(text=columns[i]).find_previous('input')
            xpath = xpath_soup(soup_element)
            driver.find_element_by_xpath(xpath).click()
        except:
            print('"{}" not found'.format(columns[i]))
            pass

In [267]:
check_boxes(soup)

In [254]:
#this may result in some leak-over of columns not explicitly specified in 'columns'
#list. This is due to the cases where the column is not one of the checkbox options

for i in range(len(columns)):
    try:
        soup_element = soup.find(text=columns[i]).find_previous('input')
        xpath = xpath_soup(soup_element)
        check_box = driver.find_element_by_xpath(xpath).click()
    except:
        print('"{}" not found'.format(columns[i]))
        pass

In [242]:
x_button = driver.find_element_by_xpath('//div/button[@aria-label="Close"]').click()

In [209]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [210]:
#regex credit: Brian Balzar from stackoverflow
#https://stackoverflow.com/questions/53014806/using-regex-to-extract-from-second-period-to-end-of-a-string
import re

headers_html = soup.find_all('th')
header = []
for item in table_headers:
    string = item.text
    try:
        subject = re.match('^([^.]+)\.([^.]+)\.(.+)$', string).group(3)
        subject = subject.replace('.', '_')
    except:
        subject = string
    header.append(subject)

In [211]:
#reading the table
table = soup.find('table')
rows = [row for row in table.find_all('tr')][1:]

data = []
for row in rows:
    row_data = defaultdict()
    for i in range(len(header)):
        row_data[header[i]] = row.find_all('td')[i].text
    data.append(row_data)

In [213]:
import csv
with open('2013-12-30-wfb.csv', 'w', newline='') as write_obj:
    # create writer object
    dict_writer = csv.DictWriter(write_obj, fieldnames=header)
    dict_writer.writeheader()
    dict_writer.writerows(data)

In [4]:
#validate
import pandas as pd
pd.read_csv('2014-12-29_data.csv')

,country,budget_date,budget_expenditures_units,budget_expenditures_value,budget_revenues_units,budget_revenues_value,budget_surplus_or_deficit_date,budget_surplus_or_deficit_percent_of_gdp,exports_partners_by_country,exports_partners_date,...,electricity_by_source_fossil_fuels_date,electricity_by_source_fossil_fuels_percent,electricity_by_source_hydroelectric_plants_date,electricity_by_source_hydroelectric_plants_percent,electricity_by_source_nuclear_fuels_date,electricity_by_source_nuclear_fuels_percent,electricity_by_source_other_renewable_sources_date,electricity_by_source_other_renewable_sources_percent,literacy_total_population_value,population_total
0,World,2013.0,USD,"24,600,000,000,000",USD,"22,410,000,000,000",2013.0,-3.1,NaN,NaN,...,2010.0,66.6,2010.0,18.8,2010.0,7.4,2010.0,7.1,84.1,"7,174,611,584"
1,Afghanistan,2012.0,USD,"4,122,000,000",USD,"2,333,000,000",2012.0,-8.7,"[{""name"":""Pakistan"",""percent"":32.2},{""name"":""I...",2012.0,...,2010.0,23.5,2010.0,76.5,2010.0,0.0,2010.0,0.0,28.1,"31,822,848"
2,Akrotiri,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"15,700"
3,Albania,2013.0,USD,"3,858,000,000",USD,"3,074,000,000",2013.0,-6.1,"[{""name"":""Italy"",""percent"":51.1},{""name"":""Spai...",2012.0,...,2012.0,5.6,2012.0,94.4,2012.0,0.0,2012.0,0.0,96.8,"3,020,209"
4,Algeria,2013.0,USD,"85,580,000,000",USD,"80,550,000,000",2013.0,-2.3,"[{""name"":""Italy"",""percent"":16},{""name"":""US"",""p...",2012.0,...,2010.0,97.5,2010.0,2.5,2010.0,0.0,2010.0,0.0,72.6,"38,813,722"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Western Sahara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2010.0,100.0,2010.0,0.0,2010.0,0.0,2010.0,0.0,NaN,"554,795"
255,Yemen,2013.0,USD,"12,310,000,000",USD,"7,769,000,000",2013.0,-10.3,"[{""name"":""China"",""percent"":41},{""name"":""Thaila...",2013.0,...,2010.0,100.0,2010.0,0.0,2010.0,0.0,2010.0,0.0,65.3,"26,052,966"
256,Zambia,2013.0,USD,"6,687,000,000",USD,"4,814,000,000",2013.0,-8.4,"[{""name"":""China"",""percent"":43.4},{""name"":""Sout...",2012.0,...,2010.0,0.4,2010.0,99.6,2010.0,0.0,2010.0,0.0,61.4,"14,638,505"
257,Zimbabwe,2013.0,NaN,NaN,NaN,NaN,NaN,NaN,"[{""name"":""China"",""percent"":21.1},{""name"":""Sout...",2012.0,...,2010.0,66.6,2010.0,33.4,2010.0,0.0,2010.0,0.0,83.6,"13,771,721"


259

In [215]:
driver.close()

In [216]:
os.path.abspath('2020-05-04_factbook.json')

'/Users/emilyng/Documents/chi20_ds15/curriculum/project-02/2020-05-04_factbook.json'

In [ ]:
file = 